<a href="https://colab.research.google.com/github/Cesare-Caputo/Tote/blob/main/image_recognition_Article_Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Create dataset

In [ ]:
import os
import cv2
import random
import numpy
import sys

# # if __name__ == "__main__":

# dict = {'Blouses': 0, #Set the name of each category and the corresponding label, the name needs to be consistent with the folder name
#         'Cardigans': 1,
#         'Coats':2,
#         'Dresses':3,
#         'Jackets':4,
#         'Jeans':5,
#         'Pants':6,
#         'Parkas':7,
#         'Polo':8,
#         'Shirts':9,
#         'Sweatshirts':10,
#         'Shorts':11,
#         'Skirts':12,
#         'Sweaters':13,
#         'Sweatpants':14,
#         'T-shirts':15}
# rate = 0.1       #Randomly draw 10% of the samples as the validation set
# root = '/content/drive/My Drive/Article Type'

# Trainlist = []
# Testlist = []
# alllist = []
# index = 0
# # max_num = 80000

# for folder in dict:
# #     img_list = [f for f in os.listdir(os.path.join(root, folder)) if not f.startswith('.')]
#     for img in img_list:
#         str0 = '%d\t%s\t%d\n' % (index, os.path.join(folder, img), dict[folder])
#         index += 1
#         alllist.append(str0)

# random.seed(100)
# random.shuffle(alllist)

# num = int(len(alllist) * rate)
# Testlist = alllist[0:num]
# Trainlist = alllist[num:]

# Trainfile = open("/content/drive/My Drive/Article Type/train.txt", "w")
# for str1 in Trainlist:
#     Trainfile.write(str1)
# Trainfile.close()

# Testfile = open("/content/drive/My Drive/Article Type/valid.txt", "w")
# for str1 in Testlist:
#     Testfile.write(str1)
# Testfile.close()

In [ ]:
# import os
# from PIL import Image
# from torch.utils import data
# import numpy as np
# from torchvision import transforms as T
# import torch


# class Fahion(data.Dataset):

#     def __init__(self, root, lists, transforms=None, train=True, test=False):

#         self.test = test
#         # imgs = [os.path.join(root, img) for img in os.listdir(root)]

#         with open (lists, 'r') as f:
#             lines = f.readlines()

#         imgs = []
#         labels = []

#         for line in lines:
#             imgs.append(os.path.join(root, line.split()[1]))
#             labels.append(line.split()[2])

#         self.imgs = imgs
#         self.labels = labels

#         if transforms is None:

#             self.transforms = T.Compose([
#                 # T.Resize(224),  # 缩放图片Zoom the image, keep the aspect ratio unchanged, the shortest side is 224 pixels
#                 T.Resize((227, 227)),  # zoom Image to (h,w)
#                 # T.RandomHorizontalFlip(), #水平翻转，注意不是所有图片都适合，比如车牌
#                 # T.CenterCrop(224),  # 从图片中间切出224*224的图片
#                 #T.RandomCrop(224),  #随机裁剪
#                 T.ToTensor(),  # 将图片(Image)转成Tensor，归一化至[0, 1]
#                 T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  # 标准化至[-1, 1]，规定均值和标准差
#             ])
#         else:
#             self.transforms = transforms


#     def __getitem__(self, index):
#         """
#         一次返回一张图片的数据
#         """
#         img_path = self.imgs[index]
#         label = self.labels[index]
#         data = Image.open(img_path)
#         data = self.transforms(data)
#         return data, label

#     def __len__(self):
#         return len(self.imgs)


# if __name__ == '__main__':

#     dataset = Fahion('/content/drive/My Drive/Article Type', '/content/drive/My Drive/Article Type/train.txt')
#     img, label = dataset[0]  # 相当于调用dataset.__getitem__(0)
#     # for img, label in dataset:
#     #     print(img.size(), img.float().mean(), label)

2. Build DenseNet

In [ ]:
"""
	The Network Class
"""

import torch
import torch.nn as nn
import math

class Dense_Block(nn.Module):
	def __init__(self, in_channels):
		super(Dense_Block, self).__init__()

		self.relu = nn.ReLU(inplace = True)
		self.bn = nn.BatchNorm2d(num_features = in_channels)

		self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
		self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
		self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
		# self.conv4 = nn.Conv2d(in_channels = 96, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
		# self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
##reduce convolution layer

	def forward(self, x):

		bn = self.bn(x)
		conv1 = self.relu(self.conv1(bn))

		conv2 = self.relu(self.conv2(conv1))
		c2_dense = self.relu(torch.cat([conv1, conv2], 1))

		conv3 = self.relu(self.conv3(c2_dense))
		c3_dense = self.relu(torch.cat([conv1, conv2, conv3], 1))

		# conv4 = self.relu(self.conv4(c3_dense))
		# c4_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4], 1))

		# conv5 = self.relu(self.conv5(c4_dense))
		# c5_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4, conv5], 1))

		# return c5_dense
		return c3_dense


class Transition_Layer(nn.Module):
	def __init__(self, in_channels, out_channels):
		super(Transition_Layer, self).__init__()

		self.relu = nn.ReLU(inplace = True)
		self.bn = nn.BatchNorm2d(num_features = out_channels)
		self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, kernel_size = 1, bias = False)
		self.avg_pool = nn.AvgPool2d(kernel_size = 2, stride = 2, padding = 0)

	def forward(self, x):

		bn = self.bn(self.relu(self.conv(x)))
		out = self.avg_pool(bn)

		return out


class DenseNet(nn.Module):
	def __init__(self, nr_classes):
		super(DenseNet, self).__init__()

		self.lowconv = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, padding = 3, bias = False)
		self.relu = nn.ReLU()

		# Make Dense Blocks
		self.denseblock1 = self._make_dense_block(Dense_Block, 64)
		self.denseblock2 = self._make_dense_block(Dense_Block, 128)
		self.denseblock3 = self._make_dense_block(Dense_Block, 128)

		# Make transition Layers
		self.transitionLayer1 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 128)
		self.transitionLayer2 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 128)
		self.transitionLayer3 = self._make_transition_layer(Transition_Layer, in_channels = 160, out_channels = 64)

		# Classifier
		self.bn = nn.BatchNorm2d(num_features = 64)
	#50176 , 512
		self.pre_classifier = nn.Linear(64*3*3, 512)
		self.classifier = nn.Linear(512, nr_classes)

	def _make_dense_block(self, block, in_channels):
		layers = []
		layers.append(block(in_channels))
		return nn.Sequential(*layers)

	def _make_transition_layer(self, layer, in_channels, out_channels):
		modules = []
		modules.append(layer(in_channels, out_channels))
		return nn.Sequential(*modules)

	def forward(self, x):
		out = self.relu(self.lowconv(x))
		print(out.shape)

		out = self.denseblock1(out)
		out = self.transitionLayer1(out)
		print(out.shape)

		out = self.denseblock2(out)
		out = self.transitionLayer2(out)
		print(out.shape)

		out = self.denseblock3(out)
		out = self.transitionLayer3(out)
		print(out.shape)

		out = self.bn(out)
		out = out.view(out.shape[0], -1)
		print(out.shape)

		out = self.pre_classifier(out)
		out = self.classifier(out)

		return out

In [ ]:
out1 = densenet.lowconv(inputs)
print(out1.shape)
# out2 = densenet.denseblock1(out1)
# print(out2.shape)
# out3 = densenet.transitionLayer1(out2)
# print(out3.shape)
# out4 = densenet.denseblock2(out3)
# print(out3.shape)
# out5 = densenet.transitionLayer2(out4)
# print(out4.shape)
# out6 = densenet.denseblock3(out5)
# print(out5.shape)
# out6 = densenet.transitionLayer3(out6)
# print(out6.shape)
# out7 = densenet.bn(out6)
# print(out7.shape)
# out8 = out7.view(out7.shape[0], -1)
# print(out8.shape)
# out9 = densenet.pre_classifier(out8)
# print(out9.shape)

NameError: ignored

3. Import and preprocess dataset

In [ ]:
import torchvision
from torchvision import datasets, models, transforms
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

dataset = torchvision.datasets.ImageFolder('/content/drive/My Drive/intern_data',data_transforms)



train_set, val_set = torch.utils.data.random_split(dataset, [10000, 2145])


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import time
import torch
import torchvision
from torch import nn, optim
from torch.utils import data
from torchvision import transforms
# from dataset import Fashion
import argparse

batch_size = 16


train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=val_set,
                                          batch_size=batch_size,
                                          shuffle=False)



NameError: ignored

4. Hyperparameters

In [ ]:
import time
import torch.optim as optim
from torch.autograd import Variable


# Hyperparameters
batch_size = 16
momentum = 0.95
learning_rate = 0.001
nr_classes = 10
nr_epochs = 10
loss_vctr = []

# Load the model on the GPU
densenet = DenseNet(nr_classes)
use_gpu = torch.cuda.is_available()
if use_gpu:
  densenet.cuda()


start = time.time()

# Oprimization Criteria and Optimization method
criterion = nn.CrossEntropyLoss().cuda() if use_gpu else nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(), lr=learning_rate, momentum=momentum, nesterov = False)


In [ ]:
!nvidia-smi

Wed Aug 12 14:34:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |   5409MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.empty_cache()

5. Training

In [ ]:
print("Start of the Optimization Processs..")
for epoch in range(nr_epochs):

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = Variable(inputs).cuda()
        labels = Variable(labels).cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = densenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            loss_vctr.append(running_loss / 2000)
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print("^^^^^^^^^^^^^^^^^")
print('Finished Optimization.')
end = time.time() # Time counted in seconds
print("The total time to train the model on the K80 GPU is : {:.1f} minutes.".format((end - start)/60))

Start of the Optimization Processs..
torch.Size([16, 50176])


RuntimeError: ignored

In [ ]:
class_correct = list(0. for i in range(16))
class_total = list(0. for i in range(16))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = densenet(images.cuda())
        _, predicted = torch.max(outputs, 1)
        c = (predicted.cpu() == labels).squeeze()
        for i in range(16):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(16):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))